In [1]:
# import carla
# import math
# import random
# import time
# import numpy as np
# import cv2
# import csv
# import threading
# import os
# import subprocess
# import sys

# # -----------------------------
# # Define the dataset directory (update path as needed)
# # -----------------------------
# dataset_dir = r"C:\Users\okeiy\OneDrive - University of Salford\Documents\Sch Notes\Dissertation\Codes\Carla_dataset"
# images_dir = os.path.join(dataset_dir, "recorded_images")
# os.makedirs(images_dir, exist_ok=True)

# # CSV file path
# csv_filename = os.path.join(dataset_dir, "control_commands.csv")
# csv_file = open(csv_filename, 'w', newline='')
# csv_writer = csv.writer(csv_file)
# csv_writer.writerow(['frame', 'timestamp', 'image_filename', 'steer', 'throttle', 'brake'])

# # -----------------------------
# # Connect to CARLA and Setup Environment
# # -----------------------------
# client = carla.Client('localhost', 2000)
# client.set_timeout(10.0)
# world = client.get_world()
# bp_lib = world.get_blueprint_library()

# # --- Start additional processes for realistic traffic ---
# # Start the traffic light controller in the background
# traffic_light_process = subprocess.Popen(['python', 'traffic_light_controller.py'])
# # Generate additional traffic: 30 vehicles and 50 walkers
# traffic_process = subprocess.Popen(['python', 'generate_traffic.py', '--number-of-vehicles', '30', '--number-of-walkers', '50'])
# time.sleep(5)

# # Set up the traffic manager
# traffic_manager = client.get_trafficmanager()
# # (Optional: adjust parameters on the traffic manager if needed)
# # traffic_manager.set_global_distance_to_leading_vehicle(2.5)

# # -----------------------------
# # Spawn the Ego Vehicle
# # -----------------------------
# spawn_points = world.get_map().get_spawn_points()
# vehicle_bp = bp_lib.find('vehicle.dodge.charger')
# vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# # Enable autopilot so the vehicle drives autonomously
# vehicle.set_autopilot(True)

# # -----------------------------
# # Spawn an RGB camera attached to the vehicle
# # -----------------------------
# camera_bp = bp_lib.find('sensor.camera.rgb')
# camera_init_trans = carla.Transform(carla.Location(x=0.1, z=1.6))
# camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# # Allow sensor to initialize
# time.sleep(0.2)

# # -----------------------------
# # Global Variables and Lock
# # -----------------------------
# frame_counter = 0
# counter_lock = threading.Lock()

# # -----------------------------
# # Graceful Shutdown Function
# # -----------------------------
# def shutdown():
#     """Stops the sensor, destroys the vehicle, closes files and windows, then exits."""
#     print("Shutting down data collection...")
#     camera.stop()
#     vehicle.destroy()
#     csv_file.close()
#     cv2.destroyAllWindows()
#     # Optionally terminate traffic processes if needed
#     traffic_light_process.terminate()
#     traffic_process.terminate()
#     sys.exit(0)

# # -----------------------------
# # Camera Callback Function for Data Collection
# # -----------------------------
# def camera_callback(image):
#     global frame_counter
#     # Convert raw data to numpy array and reshape to (height, width, 4)
#     array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
#     array = np.reshape(array, (image.height, image.width, 4))
    
#     # Convert image from RGBA to BGR (for OpenCV)
#     bgr_image = cv2.cvtColor(array, cv2.COLOR_RGBA2BGR)
    
#     # Update frame counter safely
#     with counter_lock:
#         frame_id = frame_counter
#         frame_counter += 1
        
#     # Define filename and save the image to images_dir
#     image_filename = os.path.join(images_dir, f"frame_{frame_id:06d}.png")
#     cv2.imwrite(image_filename, bgr_image)
    
#     # Get current control commands (as set by autopilot)
#     control = vehicle.get_control()
#     steer = control.steer
#     throttle = control.throttle
#     brake = control.brake
    
#     # Write data to CSV: frame id, sensor timestamp, image file, and control values
#     csv_writer.writerow([frame_id, image.timestamp, image_filename, steer, throttle, brake])
#     csv_file.flush()  # Ensure data is written immediately
    
#     # Overlay control data on the image for display
#     overlay_text = f"Steer: {steer:.2f} | Throttle: {throttle:.2f} | Brake: {brake:.2f}"
#     cv2.putText(bgr_image, overlay_text, (10, 30),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
#     # Display the image in an OpenCV window
#     cv2.imshow('RGB Camera', bgr_image)
    
#     # Check if 'q' was pressed in this frame
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         shutdown()

# # -----------------------------
# # Start the Camera Listener
# # -----------------------------
# camera.listen(camera_callback)

# # -----------------------------
# # Keep the Script Running for Data Collection
# # -----------------------------
# print("Recording data... Press 'q' in the camera window to stop.")

# # Main loop does nothing but sleep, as camera_callback handles the display.
# while True:
#     time.sleep(1)

In [2]:
import carla

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

# Retrieve the available maps
available_maps = client.get_available_maps()

# Print available maps
print("Available Maps:")
for map_name in available_maps:
    print(map_name)

Available Maps:
/Game/Carla/Maps/Mine_01
/Game/Carla/Maps/Town06
/Game/Carla/Maps/Town07
/Game/Carla/Maps/Town10HD
/Game/Carla/Maps/Town10HD_Opt


In [3]:
import carla
import time

client = carla.Client('localhost', 2000)
client.set_timeout(30.0)  # Increase the timeout significantly

# Try loading the default or an explicit world
world = client.load_world('/Game/Carla/Maps/Town06')

# Wait longer to allow the server to initialize completely.
time.sleep(5)

RuntimeError: time-out of 30000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [ ]:
import carla
import math
import random
import time
import numpy as np
import cv2
import csv
import threading
import os
import subprocess
import sys
import pygame

# -----------------------------
# Define the dataset directory (update path as needed)
# -----------------------------
dataset_dir = r"C:\Users\okeiy\OneDrive - University of Salford\Documents\Sch Notes\Dissertation\Codes\Carla_dataset"
images_dir = os.path.join(dataset_dir, "recorded_images")
os.makedirs(images_dir, exist_ok=True)

# CSV file path
csv_filename = os.path.join(dataset_dir, "control_commands.csv")
csv_file = open(csv_filename, 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame', 'timestamp', 'image_filename', 'steer', 'throttle', 'brake'])

# -----------------------------
# Connect to CARLA and Setup Environment
# -----------------------------
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
bp_lib = world.get_blueprint_library()

# --- Start additional processes for realistic traffic ---
traffic_light_process = subprocess.Popen(['python', 'traffic_light_controller.py'])
traffic_process = subprocess.Popen(['python', 'generate_traffic.py', '--number-of-vehicles', '30', '--number-of-walkers', '50'])
time.sleep(5)

traffic_manager = client.get_trafficmanager()

# -----------------------------
# Spawn the Ego Vehicle
# -----------------------------
spawn_points = world.get_map().get_spawn_points()
if not spawn_points:
    print("No spawn points available. Exiting.")
    sys.exit(1)

vehicle_bp = bp_lib.find('vehicle.dodge.charger')
vehicle = None
for spawn_point in spawn_points:
    vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)
    if vehicle is not None:
        print(f"Vehicle spawned at: {spawn_point}")
        break

if vehicle is None:
    print("Failed to spawn vehicle. Exiting.")
    sys.exit(1)

# Disable autopilot to allow for manual control
vehicle.set_autopilot(True)

# -----------------------------
# Spawn an RGB camera attached to the vehicle
# -----------------------------
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(x=0.1, z=1.6))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
time.sleep(0.2)

# -----------------------------
# Global Variables and Lock
# -----------------------------
frame_counter = 0
counter_lock = threading.Lock()

# -----------------------------
# Graceful Shutdown Function
# -----------------------------
def shutdown():
    """Stops the sensor, destroys the vehicle, closes files and windows, then exits."""
    print("Shutting down data collection...")
    camera.stop()
    vehicle.destroy()
    csv_file.close()
    cv2.destroyAllWindows()
    traffic_light_process.terminate()
    traffic_process.terminate()
    pygame.quit()
    sys.exit(0)

# -----------------------------
# Camera Callback Function for Data Collection
# -----------------------------
def camera_callback(image):
    global frame_counter
    # Convert raw data to numpy array and reshape to (height, width, 4)
    array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
    array = np.reshape(array, (image.height, image.width, 4))
    # Convert image from RGBA to BGR (for OpenCV)
    bgr_image = cv2.cvtColor(array, cv2.COLOR_RGBA2BGR)
    
    with counter_lock:
        frame_id = frame_counter
        frame_counter += 1

    image_filename = os.path.join(images_dir, f"frame_{frame_id:06d}.png")
    cv2.imwrite(image_filename, bgr_image)
    
    # Retrieve current control values (as set manually)
    control = vehicle.get_control()
    steer = control.steer
    throttle = control.throttle
    brake = control.brake
    
    csv_writer.writerow([frame_id, image.timestamp, image_filename, steer, throttle, brake])
    csv_file.flush()
    
    overlay_text = f"Steer: {steer:.2f} | Throttle: {throttle:.2f} | Brake: {brake:.2f}"
    cv2.putText(bgr_image, overlay_text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    cv2.imshow('RGB Camera', bgr_image)
    
    # If 'q' is pressed in the OpenCV window, shutdown
    if cv2.waitKey(1) & 0xFF == ord('q'):
        shutdown()

camera.listen(camera_callback)

# -----------------------------
# Initialize Pygame for Manual Control
# -----------------------------
pygame.init()
screen = pygame.display.set_mode((300, 200))
pygame.display.set_caption("Manual Control")
clock = pygame.time.Clock()

print("Recording data with manual control... Press 'q' in the camera window or close the manual control window to stop.")

# -----------------------------
# Main Loop: Process Pygame events and update vehicle control
# -----------------------------
while True:
    # Process pygame events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            shutdown()
    
    # Get keyboard inputs
    keys = pygame.key.get_pressed()
    control = carla.VehicleControl()
    # Reset control values for each frame
    control.throttle = 0.0
    control.steer = 0.0
    control.brake = 0.0
    
    # Map keys to control actions
    if keys[pygame.K_w]:
        control.throttle = 1.0  # Accelerate
    if keys[pygame.K_s]:
        control.brake = 1.0     # Brake
    if keys[pygame.K_a]:
        control.steer = -0.5    # Turn left
    if keys[pygame.K_d]:
        control.steer = 0.5     # Turn right
    
    # Apply the control to the vehicle
    if vehicle is not None:
        vehicle.apply_control(control)
    
    # Update the pygame window (if you wish to display anything)
    pygame.display.flip()
    clock.tick(30)  # Maintain roughly 30 FPS

pygame 2.2.0 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html
